In [ ]:
from google.colab import drive, userdata
import os
drive.mount('/content/drive')
drive_dir = '/content/drive/My Drive/'
data_dir = os.path.join(drive_dir, 'Detection-datasets')
cache_dir = os.path.join(drive_dir, 'model_cache')
openai_api_key = userdata.get('OPENAI_API_KEY')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q datasets
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


# TruthfulQA

In [ ]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset('truthful_qa', 'generation')
df = ds['validation'].to_pandas()
TruthfulQA_df = []
for i in range(len(df)):
    row = df.iloc[i]
    for answer in row['correct_answers']:
        TruthfulQA_df.append(pd.DataFrame([{"Statement": f"{row['question']} {answer}.", "Label": 1, 'Type': row['type'], 'Category': row['category'], 'Group': i}]))
    for answer in row['incorrect_answers']:
        TruthfulQA_df.append(pd.DataFrame([{"Statement": f"{row['question']} {answer}.", "Label": 0, 'Type': row['type'], 'Category': row['category'], 'Group': i}]))

TruthfulQA_df = pd.concat(TruthfulQA_df, axis=0).reindex()
TruthfulQA_df = TruthfulQA_df.rename({'Statement': 'Statements', 'Label': 'isTrue'}, axis=1)
TruthfulQA_df.to_csv(os.path.join(data_dir, 'TruthfulQA_prepared.csv'))

# Politicians

In [ ]:
"""import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
import numpy as np

df = pd.read_csv(os.path.join(data_dir, 'politicians_edge_substitutions.csv'))
ord = OrdinalEncoder()
groups = ord.fit_transform(np.array(df['name']).reshape(-1,1))
df['Group'] = groups.astype(np.uint32)
df = df.sort_values(by='Group', axis=0)

n_groups = 1000

selected_rows = []
for i in range(n_groups):
  sel = df[df['Group']==i]
  n_true = sel['isTrue'].sum()
  n_false = (1-sel['isTrue']).sum()
  n_sel = min(n_true, n_false)
  selected_rows.append(sel[sel['isTrue']==0][:n_sel])
  selected_rows.append(sel[sel['isTrue']==1][:n_sel])
politicians_df = pd.concat(selected_rows)
politicians_df = politicians_df.reset_index()
politicians_df.to_csv(os.path.join(data_dir, 'Politicians_prepared.csv'))
politicians_df"""

"\ndf = pd.read_csv(os.path.join(data_dir, 'politicians_edge_substitutions.csv'))\nord = OrdinalEncoder()\ngroups = ord.fit_transform(np.array(df['name']).reshape(-1,1))\ndf['Group'] = groups.astype(np.uint32)\ndf = df.sort_values(by='Group', axis=0)\n\nn_groups = 1000\n\nselected_rows = []\nfor i in range(n_groups):\n  sel = df[df['Group']==i]\n  n_true = sel['isTrue'].sum()\n  n_false = (1-sel['isTrue']).sum()\n  n_sel = min(n_true, n_false)\n  selected_rows.append(sel[sel['isTrue']==0][:n_sel])\n  selected_rows.append(sel[sel['isTrue']==1][:n_sel])\npoliticians_df = pd.concat(selected_rows)\npoliticians_df = politicians_df.reset_index()\npoliticians_df.to_csv(os.path.join(data_dir, 'Politicians_prepared.csv'))\npoliticians_df\n"

# Tegmark's datasets

In [ ]:
import os
import pandas as pd

for ds in os.listdir(os.path.join(data_dir, 'Tegmark')):
  df = pd.read_csv(os.path.join(data_dir, 'Tegmark', ds))
  ds_new_name = ds.replace('.csv', '_prepared.csv')
  df = df.rename({'statement': 'Statements', 'label': 'isTrue'}, axis=1)
  df.to_csv(os.path.join(data_dir, ds_new_name))

# Compute activations

In [ ]:
from typing import List
import transformers
from tqdm.notebook import tqdm
import numpy as np
import torch

dtype = np.float16


class Hook:
  # Inspired by https://github.com/saprmarks/geometry-of-truth/blob/main/generate_acts.py
  def __init__(self):
    self.activations = []
    self.idx = -1   # The index of the token we look at the internal state for

  def __call__(self, module, input, **kwargs):
    assert len(input) == 1 and isinstance(input[0], torch.Tensor)
    o = input[0][...,self.idx,:].detach().cpu().numpy().astype(dtype)
    self.activations.append(o)


def clear_hooks(model):
  for layer in model.model.layers:
    layer.self_attn.o_proj._forward_pre_hooks.clear()
    assert not len(layer.self_attn.o_proj._forward_pre_hooks)


def add_hooks(model):
  hooks = []
  handles = []
  for i, layer in enumerate(model.model.layers):
    hook = Hook()
    handle = layer.self_attn.o_proj.register_forward_pre_hook(hook)
    hooks.append(hook)
    handles.append(handle)
  return hooks, handles


def compute_activations(statements: List[str], model: torch.nn.Module, tokenizer) -> np.ndarray:
  """
    Returns:
    - Activations of shape [num_samples, num_layers, num_heads, head_dim]
  """
  # Empty forward hooks just in case something happened.
  clear_hooks(model)
  hooks, handles = add_hooks(model)

  for statement in tqdm(statements, leave=False, desc="Computing activations for each statement"):
    tokens = tokenizer.encode(statement, return_tensors='pt').cuda()    # Wait... How will this end up looking? <inst> Question answer </inst>? Should be <inst> Question </inst> answer
    _ = model(tokens)

  for handle in handles:
    handle.remove()

  activations = []
  for hook in hooks:
    activations.append(np.vstack(hook.activations))
  activations = np.stack(activations, axis=0)                                   # [num_layers, num_samples, n_hidden_dim]

  num_layers = len(model.model.layers)
  num_samples = len(statements)
  num_heads = model.model.layers[0].self_attn.num_heads
  head_dim = model.model.layers[0].self_attn.head_dim
  out = activations.reshape(num_layers, num_samples, num_heads, head_dim)       # [num_layers, num_samples, num_heads, head_dim]
  out = out.transpose(1,0,2,3)                                                  # [num_samples, num_layers, num_heads, head_dim]
  assert (out[2,4,3,:] == activations[4,2,(3*head_dim):(4*head_dim)]).all()
  return out                                                                    # [num_layers, num_samples, num_heads, head_dim]

In [ ]:
import torch, gc
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from typing import List, Tuple
import pandas as pd
from tqdm.notebook import tqdm

def compute_activations_on_datasets(model_name: str, datasets: List[Tuple[str, pd.DataFrame]], model_setup_fn=None):
  """
    Should traverse datasets, not models (so we don't have to re-load the same model multiple times - this is expensive)
  """

  # Load 4 bit quantized model
  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype=torch.float16,
  )
  model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", cache_dir=cache_dir, quantization_config=quantization_config, trust_remote_code=True)
  tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
  if model_setup_fn is not None:
    model, tokenizer = model_setup_fn(model, tokenizer)

  # Compute activations and save
  for dataset_name, df in tqdm(datasets, leave=False, desc="Datasets"):
    file_name_str = f"Activations_{dataset_name}_{model_name.split('/')[-1]}.npz"
    activations = compute_activations(df["Statements"], model, tokenizer)
    np.savez(os.path.join(data_dir, file_name_str), activations)

    # Check if correctly saved
    a_ = np.load(os.path.join(data_dir, file_name_str))['arr_0']
    assert np.all(activations == a_)

  # Clean up model
  model.cpu()
  del model
  torch.cuda.empty_cache()
  gc.collect()


#compute_activations_on_dataset("meta-llama/Meta-Llama-3-8B", "Politicians", df_concat_politicians)

In [ ]:
# Note: These chat templates are not used right now... Just added them here because they're needed for fine-tuning, hence maybe also for this task later on...
def opt_setup_fn(model, tokenizer):
  # Patch the attention heads to make it compatible with the rest (this one is needed!)
  model.model.layers = model.model.decoder.layers
  for l in model.model.layers:
    l.self_attn.o_proj = l.self_attn.out_proj
  return model, tokenizer

In [ ]:
datasets = [(ds_file_name.replace('_prepared.csv', ''), pd.read_csv(os.path.join(data_dir, ds_file_name))) for ds_file_name in filter(lambda x: '.csv' in x, os.listdir(data_dir))]
models = ["facebook/opt-125m", "facebook/opt-350m", "facebook/opt-2.7b", "meta-llama/Llama-2-7b-chat-hf", "microsoft/Phi-3-mini-4k-instruct", "meta-llama/Meta-Llama-3-8B"]
model_setup_fns = [opt_setup_fn, opt_setup_fn, opt_setup_fn, None, None, None]

for model, model_setup_fn in tqdm(list(zip(models, model_setup_fns)), leave=False, desc="Models"):
  # Remember model setup function!
  ## For opt: Set model.model.layers = model.model.decoder.layers
  compute_activations_on_datasets(model, datasets, model_setup_fn=model_setup_fn)

Models:   0%|          | 0/6 [00:00<?, ?it/s]

Datasets:   0%|          | 0/5 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/2422 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/5918 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/1496 [00:00<?, ?it/s]

...

Computing activations for each statement:   0%|          | 0/2422 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/5918 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/1496 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/4450 [00:00<?, ?it/s]

Computing activations for each statement:   0%|          | 0/1496 [00:00<?, ?it/s]

In [ ]:
# Load 4 bit quantized model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", device_map="auto", cache_dir=cache_dir, quantization_config=quantization_config, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B", cache_dir=cache_dir)
#model, tokenizer = opt_setup_fn(model, tokenizer)

In [ ]:
tokenizer.decode(tokenizer.encode("What happens to you if you eat watermelon seeds?"))

In [ ]:
tokenizer.apply_chat_template([
    {'role': 'user', 'content': "What happens to you if you eat watermelon seeds?"},
    {'role': 'assistant', 'content': 'Nothing happens.'}
], return_tensors='pt')

In [ ]:
tokenizer.decode(tokenizer.apply_chat_template([
    {'role': 'user', 'content': "What happens to you if you eat watermelon seeds?"},
    {'role': 'assistant', 'content': 'Nothing happens.'}
], return_tensors='pt')[0])

In [ ]:
model(tokenizer.apply_chat_template([
    {'role': 'user', 'content': "What happens to you if you eat watermelon seeds?"},
    {'role': 'assistant', 'content': 'Nothing happens.'}
], return_tensors='pt').cuda())

In [ ]:
datasets